# Проверка модели
## Метрики
Для задач бинарной классификации существует огромный набор метрик
Главная метрика в задачах классификация это *accuracy*, она же *точность*. Вычисляется как количество правильных ответов к общей доле ответов.


## Cross-validation
Наиболее популярным и надёжным способом проверки адекватности модели является кросс-валидация. Для этого датасет разбивается на $N$ частей. После разбиения проводят $N$ итераций обучения модели, фиксируя $i$-ую часть в качестве *валидационной выборки*, а *остальные* данные используя в качестве *обучающей выборки* после чего усредняют показатели точности.


# Датасет Iris (Ирисы Фишера)

Ирисы Фишера - ставший классическим датасет для проверки алгоритмов классификации. Он содержит в себе данные о 150 экземлярах цветков ириса трёх видов - Iris Setosa, Iris Verginicam Iris Versicolor. По 50 записей на вид. Записи представляют собой информацию о принадлежности к виду (класс) и четыре признака:
* sepal length - длина чашелистика, см
* sepal width - ширина чашелистика, см
* petal length - длина лепестка, см
* petal width - ширина лепестка, см

Проверим, как хорошо покажет себя метод kNN на этом наборе данных при помощи кросс-валидации.

Сделаем подготовительные импорны

*Дисклеймер: крайне рекомендуется выполнять следующие ячейки по порядку*

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np

Посмотрим, что содержит в себе датасет iris:

In [ ]:
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
def kNN_classifier(x, X_train, Y_train, k=5):
    distances = np.sqrt(np.sum((X_train - np.array(x).reshape(1, -1))** 2, axis=1))
    rank = np.argsort(distances)
    k_neighbours = Y_train[rank[:k]]

    estimates = {}

    for label in np.unique(k_neighbours):
        estimates[label] = np.sum(k_neighbours == label) / k

    estimates_rank = sorted(estimates, key=lambda label: estimates[label], reverse=True)
    
    return estimates_rank[0]

In [ ]:
iris = iris.sample(frac=1)

X = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
Y = iris['species'].values

N = 5
bounds = np.linspace(0, iris.shape[0] - 1, N+1, endpoint=True).astype(int)
for low, high in zip(bounds[:-1], bounds[1:]):
    test_mask = np.zeros_like(Y, dtype=bool)
    test_mask[low:high] = True
    train_mask = ~test_mask
    
    X_test, Y_test = X[test_mask, :], Y[test_mask]
    X_train, Y_train = X[train_mask, :], Y[train_mask]
    
    predictions = np.array([kNN_classifier(x, X_train, Y_train, k=10) for x in X_test])